# 🎨 CNN-Based Image Watermarking using DWT
## Google Colab - Optimized Version

This notebook trains and evaluates a deep learning watermarking system.

**Features:**
- Invisible watermark embedding
- Robust against 7 attack types
- Automatic evaluation with metrics
- GPU accelerated training

---

## 📦 Step 1: Setup Environment

In [ ]:
# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", "✓ YES" if tf.config.list_physical_devices('GPU') else "✗ NO")
print("\n⚠️ If GPU is not available, go to: Runtime → Change runtime type → GPU")

In [ ]:
# Clone repository (if not already cloned)
import os
if not os.path.exists('Watermarking-cnn'):
    !git clone https://github.com/Mehulsri07/Watermarking-cnn.git
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

# Change to project directory
%cd Watermarking-cnn
print("\n📁 Current directory:", os.getcwd())

In [ ]:
# Install dependencies
!pip install -q tensorflow-wavelets opencv-python scikit-image matplotlib
print("✓ Dependencies installed")

## 📥 Step 2: Download Sample Images

In [ ]:
# Download sample images from Lorem Picsum
import urllib.request
import os

def download_samples(num_train=20, num_test=5):
    os.makedirs('train_images', exist_ok=True)
    os.makedirs('test_images', exist_ok=True)
    
    print(f"Downloading {num_train} training images...")
    for i in range(num_train):
        url = f"https://picsum.photos/256/256?random={i}"
        urllib.request.urlretrieve(url, f"train_images/train_{i:03d}.jpg")
        if (i+1) % 5 == 0:
            print(f"  Downloaded {i+1}/{num_train}")
    
    print(f"\nDownloading {num_test} test images...")
    for i in range(num_test):
        url = f"https://picsum.photos/256/256?random={100+i}"
        urllib.request.urlretrieve(url, f"test_images/test_{i:03d}.jpg")
    
    print(f"\n✓ Downloaded {num_train} training + {num_test} test images")

download_samples(num_train=20, num_test=5)

## ⚙️ Step 3: Configure Training

In [ ]:
# Training configuration
EPOCHS = 10          # Number of training epochs (increase for better results)
BATCH_SIZE = 2       # Batch size (increase if you have more GPU memory)
LEARNING_RATE = 0.001

print("Training Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Learning Rate: {LEARNING_RATE}")
print("\n💡 Tip: Increase EPOCHS to 20-50 for better accuracy")

## 🚀 Step 4: Train Model

In [ ]:
# Run training and evaluation
!python train_and_evaluate.py

## 📊 Step 5: View Results

In [ ]:
# Display summary metrics
from IPython.display import Image, display
import json

# Show summary chart
print("📊 Performance Summary:")
display(Image('config_1_baseline/evaluation_results/summary_metrics.png'))

In [ ]:
# Show detailed results
with open('config_1_baseline/evaluation_results/evaluation_report.json', 'r') as f:
    report = json.load(f)

print("\n📋 Detailed Results:\n")
print("="*80)
for attack_name, stats in report['attack_statistics'].items():
    print(f"\n{attack_name}:")
    print(f"  PSNR: {stats['avg_psnr']:.2f} dB")
    print(f"  SSIM: {stats['avg_ssim']:.4f}")
    print(f"  BER:  {stats['avg_ber']:.2f}%")
print("\n" + "="*80)

In [ ]:
# Display sample visualizations
import glob

print("\n🖼️ Sample Visualizations:\n")
image_files = glob.glob('config_1_baseline/evaluation_results/images/*.png')[:3]

for img_file in image_files:
    print(f"\n{os.path.basename(img_file)}:")
    display(Image(img_file, width=800))

## 💾 Step 6: Download Results

In [ ]:
# Create zip file with all results
!zip -r results.zip config_1_baseline/

# Download
from google.colab import files
files.download('results.zip')

print("✓ Results downloaded!")

## 🧪 Step 7: Test Custom Image (Optional)

In [ ]:
# Upload your own image
from google.colab import files
import cv2
import numpy as np
from IPython.display import Image as IPImage, display

print("Upload an image to test watermarking:")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"\n✓ Uploaded: {filename}")
    
    # Process image
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=-1)
    
    # Generate watermark
    watermark = np.random.randint(0, 2, size=(256,)).astype(np.float32)
    
    # Load model
    from models.wavetf_model import WaveTFModel
    wavetf_model = WaveTFModel(image_size=(256, 256, 1), watermark_size=(256,))
    model = wavetf_model.get_model()
    model.load_weights('config_1_baseline/final_model_weights.h5')
    
    # Embed watermark
    img_batch = np.expand_dims(img, axis=0)
    wm_batch = np.expand_dims(watermark, axis=0)
    attack_batch = np.array([[0]], dtype=np.int32)
    
    watermarked, extracted = model.predict([img_batch, wm_batch, attack_batch])
    
    # Save and display
    cv2.imwrite('watermarked_output.png', (watermarked[0].squeeze() * 255).astype(np.uint8))
    
    print("\n📸 Results:")
    display(IPImage('watermarked_output.png'))
    
    # Calculate metrics
    from utils.metrics import calculate_psnr, calculate_ssim, calculate_ber
    psnr = calculate_psnr(img_batch, watermarked)
    ssim = calculate_ssim(img_batch, watermarked)
    ber = calculate_ber(wm_batch, extracted)
    
    print(f"\n📊 Metrics:")
    print(f"  PSNR: {psnr:.2f} dB")
    print(f"  SSIM: {ssim:.4f}")
    print(f"  BER:  {ber:.2f}%")

---
## ✅ Done!

Your watermarking model is trained and evaluated. Check the results above!

**Next Steps:**
- Increase EPOCHS for better accuracy
- Add more training images
- Test with your own images
- Download the trained model

**Questions?** Open an issue on GitHub!